In [1]:
import math
import itertools
import pytz
import os
import json
import pandas as pd
import numpy as np                       #for np.array
import time                              #for time.sleep

import TriPhaseController as tpc

from IPython.display import clear_output
from IPython.display import display
from pathlib import Path
%matplotlib inline

In [2]:
weather_key = '49ed35fbd71aca46756c7d5ef369ce11'

In [3]:
from pyowm.owm import OWM
from pyowm.utils import timestamps
from datetime import datetime, timedelta, timezone
from pyowm.utils import timestamps, formatting 

def get_current_weather(key, future_hours):
    
    """ Arg:
        - key(str): key get from owm site
        - future_hours(int): number of future hours
   
    Returns:
        - weather : dataframe with temperature,cloud and wind data of future hours
    """
    owm = OWM(key)
    mgr = owm.weather_manager()
    one_call_future = mgr.one_call(lat=51.98787601885725, lon=5.950209138832937)
    

    future_weather = pd.DataFrame(columns = ['temperature','cloud','wind','rain'])

    for i in range(len(one_call_future.forecast_hourly)):

        temp        = one_call_future.forecast_hourly[i].ref_time
        temp        = datetime.fromtimestamp(temp).strftime('%Y-%m-%d %H:%M:%S')
        #print(temp)
        cloud       = one_call_future.forecast_hourly[i].clouds
        cloud       = np.rint(cloud*8/100)
        temperature = one_call_future.forecast_hourly[i].temperature('celsius')['temp']
        wind        = one_call_future.forecast_hourly[i].wind()['speed']

        if one_call_future.forecast_hourly[i].status == 'rain' or one_call_future.forecast_hourly[i].status == 'Rain':
            rain = 1
        else: rain = 0

        future_weather = future_weather.append({'temperature': temperature,'cloud': cloud,
                                                'wind': wind,'rain': rain},ignore_index=True)
        
    future_weather = future_weather[1:future_hours+1]
    return future_weather

In [4]:
get_current_weather(weather_key, 1)

<ipython-input-3-edc95a13162d>:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  future_weather = future_weather.append({'temperature': temperature,'cloud': cloud,
<ipython-input-3-edc95a13162d>:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  future_weather = future_weather.append({'temperature': temperature,'cloud': cloud,
<ipython-input-3-edc95a13162d>:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  future_weather = future_weather.append({'temperature': temperature,'cloud': cloud,
<ipython-input-3-edc95a13162d>:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  future_weather = future_weather.append({'temperature': temperature,'clo

<ipython-input-3-edc95a13162d>:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  future_weather = future_weather.append({'temperature': temperature,'cloud': cloud,


,temperature,cloud,wind,rain
1,20.52,7.0,2.86,0.0


In [5]:
import paho.mqtt.client as mqtt
import paho.mqtt.publish as publish
import paho.mqtt.subscribe as subscribe

In [6]:
def callback_sample(SOC, PV_arr, EV_arr):
    # Do something here
    I = 6.66
    return [I, I+1.11, I+2.22, I+3.33, I+2.22, I+1.11]

In [7]:
test_model = tpc.Controller()
test_model.set_advanced_callback(callback_sample)
# TODO Implement set_demo_callback() for demo mode
#test_model.start_model()
#test_model.m.stop()

Select(description='Select one of the available Triphase realtime targets: ', options={}, value=None)

IndexError: list index out of range

In [ ]:
testmode_topic = "HANevse/control/testmode"
data_topic = "HANevse/control/data"

his_PV_topic = "HANevse/history/PV"
his_EV_topic = "HANevse/history/EV"

SOC_topic = "HANevse/predict/SOC"
I_topic = "HANevse/predict/I"
weather_topic = "HANevse/predict/weather"
echo_topic = "HANevse/predict/data"
I_temporary = "HANevse/predict/Itemp" #To be removed after the connection in the lab is fixed

In [ ]:
data_time = time.time()
data_arr = []

def on_message(client, userdata, message):
    global test_model
    
    global data_arr
    global data_time
    
    payload = str(message.payload.decode("utf-8"))
    topic = message.topic
    qos = message.qos
    retain = message.retain
    
    clear_output(wait=True)
    print("\t message received ", payload)
    print("\t message topic=", topic)
    print("\t message qos=", qos)
    print("\t message retain flag=", retain)
    
    if (message.topic == testmode_topic):
        value = int(payload)
        test_model.set_demo_mode(value)
    elif (message.topic == data_topic):
        m_in = json.loads(payload)
        data_time = time.time()
        data_arr = m_in
    
        if test_model.is_demo():
            SOC_data = test_model.power_control(data_arr)
            client.publish(SOC_topic, payload=json.dumps(SOC_data), qos=0, retain=False)            
            client.publish(echo_topic, payload=json.dumps(test_model.full_data), qos=0, retain=False)

In [ ]:
broker = "broker.hivemq.com"

client = mqtt.Client("Test_machine_HAN_evse")
client.connect(broker)
client.on_message = on_message        #attach function to callback
client.subscribe("HANevse/control/#")

client.loop_start() 

In [ ]:
while True:
    time.sleep(20)
    #clear_output(wait=True)
    
    # Sent PV and EV data to draw the graph #######################################################
    his_PV = test_model.Data.read_PV(5)
    historical_PV = []
    for i in range(len(his_PV)):
        historical_PV.append({'PV':his_PV[i]})
    client.publish(his_PV_topic, payload=json.dumps(historical_PV), qos=0, retain=False)
    
    his_EV = test_model.Data.read_EV(5)
    historical_EV = []
    for i in range(len(his_EV)):
        historical_EV.append({'EV':his_EV[i]})
    client.publish(his_EV_topic, payload=json.dumps(historical_EV), qos=0, retain=False)
    ###############################################################################################
    
    #To be removed after the connection in the lab is fixed #######################################
    I_now = test_model.m.capture_data('cs/i_out', samples = 1, decimation = 2)
    [[[I_temp]]] = list(I_now.data.values())
    client.publish(I_temporary, payload=json.dumps([{'I':I_temp}]), qos=0, retain=False)
    ###############################################################################################
    
    
    if not test_model.is_demo():
        # ADD PREDICTION HERE #####################################################################
        prediction = [{'EV': 12e3, 'PV': 10e3}, {'EV': 13e3, 'PV': 15e3}, {'EV': 14e3, 'PV': 20e3}, 
                      {'EV': 13e3, 'PV': 10e3}, {'EV': 10e3, 'PV': 5e3}]
        ###########################################################################################
        
        I_arr = test_model.power_control(prediction)
        
        ############# Input I to triphase control #################################################
        #
        #
        ###########################################################################################
        
        I_sent = []
        weather_sent = []
        for row in I_arr:
            I_sent.append({'I':row})
        
        weather_forecast = get_future_weather(weather_key,1)
        weather_sent.append(weather_forecast.iloc[0].to_dict())
        weather_sent[0]['I'] = I_sent[0]['I']
        
        client.publish(I_topic, payload=json.dumps(I_sent), qos=0, retain=False)
        client.publish(weather_topic, payload=json.dumps(weather_sent), qos=0, retain=False)
        client.publish(echo_topic, payload=json.dumps(test_model.full_data), qos=0, retain=False)
        client.publish(SOC_topic, payload=json.dumps([{'SOC':test_model.SOC}]), qos=0, retain=False)  

In [ ]:
client.loop_stop()

In [ ]:
test_model.m.stop()